In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import time
import numpy as np

In [2]:
df1 = pd.read_csv("/Users/iclon/Downloads/meneame_scraped_final.csv", encoding="utf-8")
df2 = pd.read_csv("/Users/iclon/Downloads/meneame_scraped_final_2.csv", encoding="utf-8")
df3 = pd.read_csv("/Users/iclon/Downloads/meneame_scraped_final_3.csv", encoding="utf-8")

In [3]:
df = pd.concat([df1,df2,df3], ignore_index=True)


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374282 entries, 0 to 374281
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   news_id          374282 non-null  int64 
 1   title            374282 non-null  object
 2   content          374265 non-null  object
 3   meneos           374282 non-null  int64 
 4   clicks           374282 non-null  int64 
 5   karma            374282 non-null  int64 
 6   positive_votes   374282 non-null  int64 
 7   anonymous_votes  374282 non-null  int64 
 8   negative_votes   374282 non-null  int64 
 9   category         374282 non-null  object
 10  comments         374282 non-null  int64 
 11  published_date   374282 non-null  object
 12  author           287091 non-null  object
 13  source           374282 non-null  object
 14  source_link      374282 non-null  object
 15  provincia        374282 non-null  object
 16  comunidad        374282 non-null  object
 17  scraped_da

In [5]:
df["published_date"] = pd.to_datetime(df["published_date"])
df["scraped_date"] = pd.to_datetime(df["scraped_date"])

In [6]:
df = df.drop_duplicates(subset=["news_id"])

# Procesamiento de 'comunidad'

In [7]:
# Diccionario de provincias-ccaa
PROVINCIAS_COMUNIDADES = {
    "Andalucía": {
        "provincias": ["Almería", "Cádiz", "Córdoba", "Granada", "Huelva", "Jaén", "Málaga", "Sevilla"]
    },
    "Aragón": {
        "provincias": ["Huesca", "Teruel", "Zaragoza"]
    },
    "Asturias": {
        "provincias": ["Asturias"]
    },
    "Islas Baleares": {
        "provincias": ["Illes Balears"],
        "equivalencias": {
            "Mallorca": "Illes Balears",
            "Menorca": "Illes Balears",
            "Cabrera": "Illes Balears",
            "Ibiza": "Illes Balears",
            "Formentera": "Illes Balears"
        }
    },
    "Canarias": {
        "provincias": ["Las Palmas", "Santa Cruz de Tenerife"],
        "equivalencias": {
            "La Gomera": "Canarias",
            "El Hierro": "Canarias",
            "La Graciosa": "Canarias",
            "La Palma": "Canarias",
            "Gran Canaria": "Canarias",
            "Tenerife": "Canarias",
            "Lanzarote": "Canarias",
            "Fuerteventura": "Canarias"
        }
    },
    "Cantabria": {
        "provincias": ["Cantabria"]
    },
    "Castilla La Mancha": {
        "provincias": ["Albacete", "Ciudad Real", "Cuenca", "Guadalajara", "Toledo"]
    },
    "Castilla y León": {
        "provincias": ["Ávila", "Burgos", "León", "Palencia", "Salamanca", "Segovia", "Soria", "Valladolid", "Zamora"]
    },
    "Cataluña": {
        "provincias": ["Barcelona", "Girona", "Lleida", "Tarragona"],
        "equivalencias": {
            "Gerona": "Girona",
            "Lérida": "Lleida"
        }
    },
    "Extremadura": {
        "provincias": ["Badajoz", "Cáceres"]
    },
    "Galicia": {
        "provincias": ["A Coruña", "Lugo", "Ourense", "Pontevedra"],
        "equivalencias": {
            "Coruña": "A Coruña"
        }
    },
    "Madrid": {
        "provincias": ["Madrid"]
    },
    "Murcia": {
        "provincias": ["Murcia"]
    },
    "Navarra": {
        "provincias": ["Navarra"]
    },
    "La Rioja": {
        "provincias": ["La Rioja"]
    },
    "País Vasco": {
        "provincias": ["Araba/Álava", "Bizkaia/Vizcaya", "Gipuzkoa/Guipúzcoa"],
        "equivalencias": {
            "Araba": "Araba/Álava",
            "Álava": "Araba/Álava",
            "Bizkaia": "Bizkaia/Vizcaya",
            "Vizcaya": "Bizkaia/Vizcaya",
            "Gipuzkoa": "Gipuzkoa/Guipúzcoa",
            "Guipúzcoa": "Gipuzkoa/Guipúzcoa"
        }
    },
    "Comunidad Valenciana": {
        "provincias": ["Alicante/Alacant", "Castelló/Castellón", "València/Valencia"],
        "equivalencias": {
            "Alicante": "Alicante/Alacant",
            "Alacant": "Alicante/Alacant",
            "Castellón": "Castelló/Castellón",
            "Castelló": "Castelló/Castellón",
            "Valencia": "València/Valencia",
            "València": "València/Valencia"
        }
    },
    "Ceuta": {
        "provincias": ["Ceuta"]
    },
    "Melilla": {
        "provincias": ["Melilla"]
    }
}

In [8]:
# Función para asignar provincias y ccaa del title/content
def asignar_provincia_comunidad(row):
    title = str(row["title"]) if pd.notna(row["title"]) else ""
    content = str(row["content"]) if pd.notna(row["content"]) else ""

    for comunidad, datos in PROVINCIAS_COMUNIDADES.items():
        provincias = datos["provincias"]
        equivalencias = datos.get("equivalencias", {})

        for provincia in provincias:
            if provincia in title or provincia in content:
                return {"provincia": provincia, "comunidad": comunidad}

        for variante, provincia_estandar in equivalencias.items():
            if variante in title or variante in content:
                return {"provincia": provincia_estandar, "comunidad": comunidad}

    return {"provincia": "Desconocido", "comunidad": "Desconocido"}

In [9]:
# Aplicar la función fila por fila
df[["provincia", "comunidad"]] = df.apply(asignar_provincia_comunidad, axis=1).apply(pd.Series)

In [10]:
df["comunidad"].value_counts()

comunidad
Desconocido             239538
Madrid                   14371
Andalucía                 6521
Comunidad Valenciana      5529
Cataluña                  5432
Castilla y León           3883
Castilla La Mancha        1585
Galicia                   1564
Aragón                    1363
Canarias                  1319
Murcia                    1232
Asturias                  1136
Islas Baleares            1036
Navarra                    539
País Vasco                 531
Extremadura                488
Cantabria                  407
Ceuta                      240
Melilla                    158
La Rioja                   146
Name: count, dtype: int64

In [11]:
df['comunidad'] = df['comunidad'].replace('Desconocido', np.nan)

df['provincia'] = df['provincia'].replace('Desconocido', np.nan)



# Procesamiento de Categorias


In [12]:
news_category_map = {
    "Geopolítica y Sociedad": ["EEUU", "MADRID", "Rusia", "Japón", "México", "Venezuela", "Cataluña", "Galiza", "Andalucía", "Balears", "Canarias", "CastillaLeón", "deutschland", "Barcelona", "Galicia", "Cantabria", "Aragón", "Málaga", "Roma", "INTERNAC", "Africa", "OrienteMedio", "Cuba"],
    "Movimientos políticos": ["politica", "corruPPción", "corruptcion", "corrupcion", "EleccionesE", "Podemos", "República", "RentaBásica", "Cayetanos", "LuchaDClases", "Anarquismo", "DDHH", "Autonomos", "Justicia7291", "Tribunales", "actualidad", "ZasSmeame", "Campañas", "Fachéame", "VIOLENCIA_G", "prensa", "borbones", "KKKantiPDMS", "AEDE", "procescatalà", "RebeliónMnm", "DchoLaboral", "Bulos", "Desinfórmame", "Madriléame", "Begoñéame"],
    "Conflictos y Guerra": ["Palestina", "Israel", "Siria", "GuerraCivilñ", "ayudaucrania", "Rescates"],
    "Tecnología y Ciencia": ["tecnología", "Informática", "GNU_LINUX", "linux", "startup", "Criptomoneda", "bitcoin", "SectorTIC", "Meteorología", "Física", "genetica", "Neurociencia", "Debian", "retrocomp", "hacking", "Metamaterial", "Micromundo", "atomo", "Impresión3D", "Drones", "netsec", "RaspberryPi", "SysDevs", "ciencia", "astronomia", "ASTRONÁUTICA", "softlibre", "Ubuntu", "android", "smartphone", "Firefox", "Blender", "IOOcia", "Marte", "Visualdata", "INVENTOS", "Numismática", "FÓSILES"],
    "Salud y Medicina": ["salud", "medicina", "Nutrición", "sanidad", "PREVENCIÓN", "HOMEOPATÍA", "VacunaGate", "vacúname", "SANIDAD", "Alimentación", "ebola", "Coronavirus"],
    "Gastronomía": ["Cachopos", "cervecéame"],
    "Negocios y Economía": ["Economía", "banca", "subvenciones", "CrisEnergéti"],
    "Entretenimiento y Cultura": ["ocio", "Cine", "Series", "Videojuegos", "Fotografía", "Música", "cinéfilos", "FolkloreÑ", "Cómics", "Webcomics", "ArteyMercado", "Arte", "Ilustracion", "clásica", "PALABROS", "Covers", "cultura", "Eurovision", "Poesia", "Bushido", "Arquitectura", "rol", "Podcasteros", "Viajar", "CiFi", "Gastronomia", "Clicómics", "Cocíname", "curiosidades", "acapela", "twitter", "reddit", "museo", "Relatocorto", "Autorrelatos", "Reportajes", "Entrevistas", "microrelatos", "Hazlotumismo", "Los12monos", "Pregúntame", "Recomiéndame", "LETREROS", "Escúchame", "Documentales", "STARWARS", "Ochenteando", "UHF", "Preguntas", "Fotomundo", "roleros", "metal", "B.S.O.", "TEMAZOS", "Comiñas", "MundoCelta", "Artículos", "sabíasque", "paléame"],
    "Deportes": ["deportes", "Natación", "Motociclismo", "ciclismo"],
    "Medioambiente y Energía": ["veganismo", "MenteAnimal", "Abandonos", "MUNDO_PROTE", "MAmbiente", "Cambioclima", "Ecología", "Energias", "sequía", "Naturaleza", "Botánica", "PLANTAS", "AgroInfo", "Vivienda", "Animales", "gatos", "Biologia", "Faros", "mascotas", "Cáñamo", "Arañas"],
    "Historia y Humanidades": ["Historia", "Mitología", "Arqueología", "Egiptologia", "Viñetas", "Hallazgos", "Asturias", "Jurídicas", "Etimologia", "Psicología", "Psicópatas", "Filosofía", "Efemérides", "Helénica", "Hemeroteca", "Fragmentos"],
    "Humor y Memes": ["Humor", "memes", "Sarcasmos", "Chorridolias", "Tendenciosos", "Ayuséame", "podríame", "Fakeame", "Fakejóo", "Idiocracia", "meneametoday", "TuMeme", "Cosasguays", "friki", "procrastrina", "NoMundoToday", "Memeneame", "Ardilléame", "Toréame", "OYOYOY", "gilipolleces", "chorradas"],
    "Transporte": ["aviacion", "aviones", "trenes", "DIRIGIBLES", "Submarino", "Motor", "Avionéame", "BARCOS", "Tractores", "movilidad", "alas"],
    "Educación": ["Educación", "idiomas", "Enseñanza", "Literatura", "Libros", "lenguas", "Matemáticas"],
    "Crimen": ["INV", "Gamonal", "Sectas", "Sucesos"],
    "Cuestiones Sociales": ["Asperger", "Crianza", "Drogolegas", "Feminismo", "LGBT", "Derecho", "SEXOLOGÍA", "mujeresenlah", "Abuso_Animal", "religion", "laicismo"]
}


In [13]:
def category_mapping(category):
    return news_category_map.get(category, "Miscelaneo")

df["category"] = df["category"].apply(category_mapping)

In [14]:
df["category"].value_counts()

category
Miscelaneo                                                                   287017
[Educación, idiomas, Enseñanza, Literatura, Libros, lenguas, Matemáticas]         1
Name: count, dtype: int64

In [15]:
df

,news_id,title,content,meneos,clicks,karma,positive_votes,anonymous_votes,negative_votes,category,comments,published_date,author,source,source_link,provincia,comunidad,scraped_date,full_story_link,user
0,4028534,Estudio realizado en Shenzhen muestra que los ...,Un programa piloto completado el año pasado en...,34,36,418,28,6,0,Miscelaneo,11,2025-02-15 01:20:02,Gry,scmp.com,https://www.scmp.com/business/china-evs/articl...,NaN,NaN,2025-02-15,NaN,NaN
1,4028533,"Detienen al ladrón de Lucky, el perro de Navat...",La investigación ha permitido denunciar penalm...,60,315,425,41,19,1,Miscelaneo,15,2025-02-15 00:45:03,Wolfgang,lavanguardia.com,https://www.lavanguardia.com/mascotas/20250213...,NaN,NaN,2025-02-15,NaN,NaN
2,4028513,"El padre de Marian Rojas, reconocido psiquiatr...","Las redes cancelan a Marian Rojas Estapé, serí...",88,294,453,53,35,1,Miscelaneo,45,2025-02-15 00:15:02,Verdaderofalso,mundodeportivo.com,https://www.mundodeportivo.com/actualidad/2025...,NaN,NaN,2025-02-15,NaN,NaN
3,4028551,"Nieves Concostrina: ""No hubo ningún San Valent...","La historia oficiosa data del siglo III, en el...",107,258,416,60,47,4,Miscelaneo,28,2025-02-14 23:55:02,Charles_Dexter_Ward,cadenaser.com,https://cadenaser.com/nacional/2025/02/14/niev...,NaN,NaN,2025-02-15,NaN,NaN
4,4028478,Ofensiva israelí contra palestinos de Cisjorda...,La agencia de la ONU para los refugiados pales...,123,130,437,58,65,0,Miscelaneo,10,2025-02-14 23:15:02,onainigo,resumen.cl,https://resumen.cl/articulos/ofensiva-israeli-...,NaN,NaN,2025-02-15,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287086,6,Instalar Linux en iPod Mini | GPL Tarragona,Enseña a instalar Linux en el iPod Mini.\n\nMu...,29,0,31,29,0,0,Miscelaneo,0,2005-12-07 12:00:01,asappla,gpltarragona.org,http://www.gpltarragona.org/node/view/395,Tarragona,Cataluña,2025-02-16,NaN,NaN
287087,4,Entrevista de El Mundo a Jose Antonio Marina,2. ¿Es la ética la filosofía del futuro?\nSi. ...,18,0,22,18,0,0,Miscelaneo,0,2005-12-07 11:30:01,gabrielperezs,el-mundo.es,http://www.el-mundo.es/encuentros/invitados/20...,NaN,NaN,2025-02-16,NaN,NaN
287088,3,Prova xfce 4.2.3!,Ara que els de repositoris de totes les distri...,34,0,13,34,0,0,Miscelaneo,0,2005-12-07 10:30:01,paurullan,xfce.org,http://www.xfce.org/release_notes/4.2.3.1_chan...,NaN,NaN,2025-02-16,NaN,NaN
287089,2,Los 84 errores de noxtrum,"Pues sí, noxtrum, el buscador de TPI y Telefón...",61,0,11,61,0,0,Miscelaneo,5,2005-12-07 09:25:02,FrIkI,pacoros.net,http://www.pacoros.net/diario/2005/12/07/los-8...,NaN,NaN,2025-02-16,NaN,NaN


In [16]:
df.to_csv("meneame_procesado.csv", index=False, encoding="utf-8")
